Implementing all the libraries we will need:

In [0]:
import sys
import sklearn as sk
import tensorflow as tf
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import numpy as np
import os
from shutil import copy, copytree
import matplotlib.pyplot as plt


# Is everything normal? Let's check.
print(sys.version)
print(keras.__version__)
print(pd.__version__)
print(tf.__version__)
print("Available" if tf.test.is_gpu_available() else "Not Available")

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
2.2.4-tf
0.25.3
1.15.0
Available


**Download and Extract the dataset**

In [0]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar xzf food-101.tar.gz

--2020-01-15 12:19:25--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2020-01-15 12:19:26--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  10.9MB/s    in 8m 20s  

2020-01-15 12:27:47 (9.53 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [0]:
!ls

food-101  food-101.tar.gz  sample_data


In [0]:
!ls food-101

images	license_agreement.txt  meta  README.txt


In [0]:
!ls food-101/images

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

**Splitting the complete dataset into train and test.**

Since it's hard to work on a dataset big like this, we will find the best parameters in small part of the dataset and then with this parameters we will work on the big dataset which has 101 class and 101.000 images.
Since the test cases which is under created before us, we decided to use this cases which are given in meta folder under food-101 folder like that:

In [0]:
!ls food-101/meta

classes.txt  labels.txt  test.json  test.txt  train.json  train.txt


We will use this train.txt and test.txt file. There are class names in them. It means there is prepared train test condition.
The goal with the prepared test-train condition is that allowing lots of people around the world race together accurately. 

In [0]:
train101_src = "food-101/train101"
test101_src = "food-101/test101"

In [0]:
with open('food-101/meta/train.txt', 'r') as file:
  picturePaths = [read.strip() for read in file.readlines()]
  for path in picturePaths:
    path = path.split('/')
    if not os.path.exists(train101_src+'/' + path[0]):
      os.makedirs(os.path.join(train101_src+'/'+path[0]))
    copy('food-101/images/'+path[0] + "/" +path[1]+'.jpg', train101_src +'/'+path[0]+"/"+path[1]+'.jpg')
print("101 class train cases created.")

101 class train cases created.


In [0]:
with open('food-101/meta/test.txt', 'r') as txt:
  picturePaths = [read.strip() for read in txt.readlines()]
  for path in picturePaths:
    path = path.split('/')
    if not os.path.exists(test101_src +'/' + path[0]):
      os.makedirs(os.path.join(test101_src +'/'+path[0]))
    copy('food-101/images/'+path[0] + "/" +path[1]+'.jpg', test101_src +'/'+path[0]+"/"+path[1]+'.jpg')
print("101 class test cases created.")

101 class test cases created.


Creating 11 class dataset to find the best parameters of GoogleNet for our purpose

In [0]:
train11_src = "food-101/train11"
test11_src = "food-101/test11"

os.makedirs(train11_src)
os.makedirs(test11_src)

smallDataset = ['baklava', 'falafel', 'hamburger', 'hot_dog', 'ice_cream', 'omelette', 'pizza', 'ramen', 'steak', 'sushi', 'waffles']

In [0]:
for className in smallDataset:
    copytree(train101_src+'/'+className,train11_src+'/'+className)
    copytree(test101_src+'/'+className,test11_src+'/'+className)
print("11 class train-test cases created.")

11 class train-test cases created.


**Next Step: Building GoogleNet Convulational Neural Network Model**

In [0]:
learning_rate = 0
batch_size = 0
epoch = 0
dropout = 0
regularizers = 0.005
class_size = 0
another_dense = none
activation = ""
cost_function = ""
train_src = ""
include_flatten = none
validation_src =""
num_of_train = 0
num_of_test = 0
learning_rate=0.0005,momentum=0.9,
inception = InceptionV3(weights='imagenet', include_top=False)

Function to graph the outputs:

In [0]:
def plot_everything(history,title,is_Activation):
  if is_Activation:
    plt.title(title)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()
  else:
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss','validation_loss'], loc='best')
    plt.show()

# **The model:**

In [0]:
def quick_train_test_graph(title='graph',batch_size=16, epoch=20, dropout=0.5, 
loss_func='categorical_crossentropy', class_size=11, learning_rate=0.0005, 
momentum=0.9, another_dense=False, include_flatten=False, img_width=299, 
img_height=299, inception=InceptionV3(weights='imagenet', include_top=False), 
activation_func='tanh'):

  if class_size == 101:
    train_src = train101_src
    test_src = test101_src
    num_of_train = 75750
    num_of_test = 25250
  if class_size == 11:
    train_src = train11_src
    test_src = test11_src
    num_of_train = 8250
    num_of_test = 7250

  train_data = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True).flow_from_directory(
      train_src,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='categorical')

  validation_data = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
    test_src,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
  
  
  inceptionv3 = InceptionV3(weights='imagenet', include_top=False)
  x = inceptionv3.output
  x = GlobalAveragePooling2D()(x)

  if(include_flatten):
    x = Flatten(name='flatten')(x)

  x = Dense(128,activation=activation_func)(x)

  if(another_dense):
    x = Dense(101,activation=activation_func)(x)
    
  x = Dropout(dropout)(x)
  predictions = Dense(class_size ,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)
  
  # Creating the model
  model = Model(inputs=inceptionv3.input, outputs=predictions)
  # Compiling the model
  model.compile(optimizer=SGD(lr=learning_rate, momentum=momentum),  loss=loss_func, metrics=['accuracy'])
  # Train phase
  history = model.fit_generator(train_data,
                      steps_per_epoch = num_of_train // batch_size,
                      validation_data = validation_data,
                      validation_steps= num_of_test // batch_size,
                      epochs=epoch,
                      verbose=1,
                      callbacks=[CSVLogger('history.log'), ModelCheckpoint(filepath='model.hdf5', verbose=1, save_best_only=True)])
  # Saving the model
  model.save('model.hdf5')
  print(history.history)
  plot_it(history,title,True)
  plot_it(history,title,False)

# **Test Phase**

**Testing Learning Rate, Momentum and Epochs**

In [0]:
quick_train_test_graph(title='0.0005lr, 10epochs, 0.9momentum',learning_rate=0.0005, epoch=10, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.0005lr, 20epochs, 0.9momentum',learning_rate=0.0005, epoch=20, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.0005lr, 20epochs, 0momentum',learning_rate=0.0005, epoch=20, momentum=0)

In [0]:
quick_train_test_graph(title='0.0001lr, 10epochs, 0.9momentum',learning_rate=0.0001, epoch=10, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.0001lr, 20epochs, 0.9momentum',learning_rate=0.0001, epoch=20, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.0001lr, 20epochs, 0momentum',learning_rate=0.0001, epoch=20, momentum=0)

In [0]:
quick_train_test_graph(title='0.001lr, 10epochs, 0.9momentum',learning_rate=0.001, epoch=10, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.001lr, 20epochs, 0.9momentum',learning_rate=0.001, epoch=20, momentum=0)

In [0]:
quick_train_test_graph(title='0.001lr, 20epochs, 0momentum',learning_rate=0.001, epoch=20, momentum=0.0)

In [0]:
quick_train_test_graph(title='0.0005lr, 10epochs, 0.9momentum',learning_rate=0.0005, epoch=10, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.0005lr, 20epochs, 0.9momentum',learning_rate=0.0005, epoch=20, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.0005lr, 20epochs, 0momentum',learning_rate=0.0005, epoch=20, momentum=0)

In [0]:
quick_train_test_graph(title='0.005lr, 10epochs, 0.9momentum',learning_rate=0.005, epoch=10, momentum=0.9)

In [0]:
quick_train_test_graph(title='0.005lr, 20epochs, 0.9momentum',learning_rate=0.005, epoch=20, momentum=0.9)

**Testing speed of batch sizes**

We also know that google colab cannot work with batch sizes higher than 16.

In [0]:
quick_train_test_graph(title='batch_size:4', batch_size=4)

In [0]:
quick_train_test_graph(title='batch_size:8',batch_size=8)

In [0]:
# quick_train_test_graph(title='batch_size:16',batch_size=16)
# This case has ran in the case above with title '0.0005lr, 20epochs, 0.9momentum'

**Testing Flatten**

In [0]:
quick_train_test_graph(title='Flatten is included',include_flatten=True)

In [0]:
# quick_train_test_graph(title='Flatten is not included',include_flatten=True)
# This case has ran in the case above with title '0.0005lr, 20epochs, 0.9momentum'

**Testing Dropout**

In [0]:
quick_train_test_graph(title='dropout:0.5',dropout=0.5)

In [0]:
quick_train_test_graph(title='dropout:0.005',dropout=0.005)

In [0]:
quick_train_test_graph(title='dropout:0.8',dropout=0.8)

In [0]:
# quick_train_test_graph(title='dropout:0',dropout=0)
# This case has ran in the case above with title '0.0005lr, 20epochs, 0.9momentum'

**Testing Loss Functions**

In [0]:
# quick_train_test_graph(title='categorical_crossentropy',activation_func='categorical_crossentropy')
# This case has ran in the case above with title '0.0005lr, 20epochs, 0.9momentum'

In [0]:
quick_train_test_graph(title='categorical_hinge',activation_func='categorical_hinge')

**Testing Activation Functions**

In [0]:
# quick_train_test_graph(title='relu',activation_func='relu')
# This case has ran in the case above with title '0.0005lr, 20epochs, 0.9momentum'

In [0]:
quick_train_test_graph(title='prelu',activation_func='prelu')

In [0]:
quick_train_test_graph(title='leakyrelu',activation_func='leakyrelu')

In [0]:
quick_train_test_graph(title='tanh',activation_func='tanh')

**Adding another Dense**

In [0]:
quick_train_test_graph(title='doubleDense',another_dense=True)
# Due to it's long process time and low accuracy, we terminated this case.